In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("Agg")
%matplotlib inline
import datetime

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys

sys.path.append("../FinRL-Library")

_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATE_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"

In [5]:
import os

if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


In [6]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [7]:
dp = YahooFinanceProcessor()
df = dp.download_data(
    start_date=_TRAIN_DATA_START, end_date=_TEST_DATA_END, ticker_list=config_tickers.DOW_30_TICKER, time_interval="1D"
)

[*********************100%***********************]  1 of 1 completed


/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/data_processors/processor_yahoofinance.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/data_processors/processor_yahoofinance.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/data_processors/processor_yahoofinance.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/data_processors/processor_yahoofinance.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/data_processors/processor_yahoofinance.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(temp_df)


Shape of DataFrame:  (88283, 9)


In [8]:
df.head()

,date,open,high,low,close,adjcp,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.515213,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,57.720001,43.267193,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,40.919998,33.792698,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,56.180000,43.777542,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,58.549999,41.353199,7325600,CAT,0


In [9]:
df.shape

(88283, 9)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [10]:
fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)

df = fe.preprocess_data(df)

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packag

Successfully added technical indicators


In [11]:
df.shape

(87580, 17)

In [12]:
df.head()

,date,open,high,low,close,adjcp,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.515213,493729600,AAPL,0,0.0,7.66851,7.631133,100.0,66.666667,100.0,7.643214,7.643214
3020,2010-01-04,56.630001,57.869999,56.560001,57.720001,43.267193,5277400,AMGN,0,0.0,7.66851,7.631133,100.0,66.666667,100.0,57.720001,57.720001
6040,2010-01-04,40.810001,41.099998,40.389999,40.919998,33.792698,6894300,AXP,0,0.0,7.66851,7.631133,100.0,66.666667,100.0,40.919998,40.919998
9060,2010-01-04,55.720001,56.389999,54.799999,56.180000,43.777542,6186700,BA,0,0.0,7.66851,7.631133,100.0,66.666667,100.0,56.180000,56.180000
12080,2010-01-04,57.650002,59.189999,57.509998,58.549999,41.353199,7325600,CAT,0,0.0,7.66851,7.631133,100.0,66.666667,100.0,58.549999,58.549999


## Add covariance matrix as states

In [13]:
# add covariance matrix as states
df = df.sort_values(["date", "tic"], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback = 252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i - lookback : i, :]
    price_lookback = data_lookback.pivot_table(index="date", columns="tic", values="close")
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({"date": df.date.unique()[lookback:], "cov_list": cov_list, "return_list": return_list})
df = df.merge(df_cov, on="date")
df = df.sort_values(["date", "tic"]).reset_index(drop=True)

In [14]:
df.shape

(80272, 19)

In [15]:
df.head()

,date,open,high,low,close,adjcp,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033259,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
1,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640533,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
2,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488434,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
3,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034512,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
4,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190567,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [16]:
train = data_split(df, _TRAIN_DATA_START, _TRAIN_DATE_END)
# trade = data_split(df, '2020-01-01', config.END_DATE)

In [17]:
train.head()

,date,open,high,low,close,adjcp,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033259,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
0,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640533,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
0,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488434,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
0,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034512,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
0,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190567,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


## Environment for Portfolio Allocation


In [18]:
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

from rl.experiments._2_ta_orig.Env import StockPortfolioEnv

In [19]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 29


In [20]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)

In [21]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [22]:
# initialize
agent = DRLAgent(env=env_train)

### Model 1: **A2C**


In [23]:
agent = DRLAgent(env=env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c", model_kwargs=A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [24]:
trained_a2c = agent.train_model(model=model_a2c, tb_log_name="a2c", total_timesteps=50000)

KeyboardInterrupt: 

In [119]:
trained_a2c.save("trained_models/trained_a2c.zip")

### Model 2: **PPO**


In [120]:
agent = DRLAgent(env=env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo", model_kwargs=PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [121]:
trained_ppo = agent.train_model(model=model_ppo, tb_log_name="ppo", total_timesteps=80000)

----------------------------------
| time/              |           |
|    fps             | 323       |
|    iterations      | 1         |
|    time_elapsed    | 6         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 2831996.5 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4298722.688411022
Sharpe:  0.8702070214816376
---------------------------------------
| time/                   |           |
|    fps                  | 303       |
|    iterations           | 2         |
|    time_elapsed         | 13        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -41.1     |
|    explained_variance   | 0         |
|    learning_rate        | 0.0001    |
|    loss                 | 4.82e+14  |
|    n_updates            | 10       

In [122]:
trained_ppo.save("trained_models/trained_ppo.zip")

### Model 3: **DDPG**


In [123]:
agent = DRLAgent(env=env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [124]:
trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name="ddpg", total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4300624.639144743
Sharpe:  0.8851489574575785
begin_total_asset:1000000
end_total_asset:4147985.3341497835
Sharpe:  0.8526189076875208
begin_total_asset:1000000
end_total_asset:4147985.3341497835
Sharpe:  0.8526189076875208
begin_total_asset:1000000
end_total_asset:4147985.3341497835
Sharpe:  0.8526189076875208
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 38        |
|    time_elapsed    | 284       |
|    total_timesteps | 11072     |
| train/             |           |
|    actor_loss      | -6.18e+07 |
|    critic_loss     | 4.52e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 8304      |
|    reward          | 4147985.2 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4147985.3341497835
Sharpe:  0.8526189076875208
begin_total_asset:1000000
end_total_asset:4147985.3341497835
Sharpe:  0.8526189076875208
begin_total_

In [125]:
trained_ddpg.save("trained_models/trained_ddpg.zip")

### Model 4: **SAC**


In [126]:
agent = DRLAgent(env=env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac", model_kwargs=SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [127]:
trained_sac = agent.train_model(model=model_sac, tb_log_name="sac", total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4242749.461920967
Sharpe:  0.8476402088468916
begin_total_asset:1000000
end_total_asset:4321699.32079811
Sharpe:  0.856508003231149
begin_total_asset:1000000
end_total_asset:4321620.39164834
Sharpe:  0.8565003886301333
begin_total_asset:1000000
end_total_asset:4321652.253116804
Sharpe:  0.8565062581807509
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 40        |
|    time_elapsed    | 270       |
|    total_timesteps | 11072     |
| train/             |           |
|    actor_loss      | -8.97e+07 |
|    critic_loss     | 4.4e+11   |
|    ent_coef        | 2.84      |
|    ent_coef_loss   | -173      |
|    learning_rate   | 0.0003    |
|    n_updates       | 10971     |
|    reward          | 4321652.5 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4321725.043240454
Sharpe:  0.8565214085748056
begin_total_asset:1000000
end_total_as

In [128]:
trained_sac.save("trained_models/trained_sac.zip")

### Model 5: **TD3**


In [129]:
agent = DRLAgent(env=env_train)
TD3_PARAMS = {"batch_size": 100, "buffer_size": 1000000, "learning_rate": 0.001}

model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [130]:
trained_td3 = agent.train_model(model=model_td3, tb_log_name="td3", total_timesteps=30000)

begin_total_asset:1000000
end_total_asset:4040495.224264071
Sharpe:  0.8284479843156909
begin_total_asset:1000000
end_total_asset:3897628.6417802908
Sharpe:  0.818323287233011
begin_total_asset:1000000
end_total_asset:3897628.6417802908
Sharpe:  0.818323287233011
begin_total_asset:1000000
end_total_asset:3897628.6417802908
Sharpe:  0.818323287233011
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 63        |
|    time_elapsed    | 174       |
|    total_timesteps | 11072     |
| train/             |           |
|    actor_loss      | -3.21e+07 |
|    critic_loss     | 2.45e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 8304      |
|    reward          | 3897628.8 |
----------------------------------
begin_total_asset:1000000
end_total_asset:3897628.6417802908
Sharpe:  0.818323287233011
begin_total_asset:1000000
end_total_asset:3897628.6417802908
Sharpe:  0.818323287233011
begin_total_asset

In [131]:
trained_td3.save("trained_models/trained_td3.zip")

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [168]:
trade = data_split(df, _TEST_DATA_START, _TEST_DATA_END)
e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

In [169]:
trade.shape

(7279, 19)

In [187]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)

begin_total_asset:1000000
end_total_asset:1192459.5479677503
Sharpe:  1.5202330773792652
hit end!


In [188]:
df_daily_return.head()

,date,daily_return
0,2021-01-04,0.000000
1,2021-01-05,0.004862
2,2021-01-06,0.014423
3,2021-01-07,0.006918
4,2021-01-08,0.001151


In [181]:
df_daily_return.to_csv("df_daily_return.csv")

In [190]:
df_actions.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2021-01-05,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118
2021-01-06,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118
2021-01-07,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118
2021-01-08,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118
2021-12-27,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118
2021-12-28,0.017701,0.017701,0.048118,0.048118,0.017701,0.017701,0.048118,0.048118,0.017701,0.048118,...,0.048118,0.017701,0.048118,0.048118,0.017701,0.048118,0.048118,0.048118,0.017701,0.048118


In [183]:
df_actions.to_csv("df_actions.csv")

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [184]:
from pyfolio import timeseries

DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func(
    returns=DRL_strat, factor_returns=DRL_strat, positions=None, transactions=None, turnover_denom="AGB"
)

/Users/zlapik/my-drive-zlapik/0-todo/Finrl-Meta/venv3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [185]:
print("==============DRL Strategy Stats===========")
perf_stats_all

==============DRL Strategy Stats===========


Annual return          0.193178
Cumulative returns     0.192342
Annual volatility      0.119302
Sharpe ratio           1.540426
Calmar ratio           2.807577
Stability              0.722255
Max drawdown          -0.068806
Omega ratio            1.292382
Sortino ratio          2.265079
Skew                  -0.285174
Kurtosis               0.635128
Tail ratio             1.078700
Daily value at risk   -0.014301
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [186]:
# baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(ticker="^DJI", start=_TEST_DATA_START, end=_TEST_DATA_END)

stats = backtest_stats(baseline_df, value_col_name="close")

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (251, 8)
Annual return          0.205174
Cumulative returns     0.204282
Annual volatility      0.124063
Sharpe ratio           1.572707
Calmar ratio           3.101403
Stability              0.770367
Max drawdown          -0.066155
Omega ratio            1.301982
Sortino ratio          2.292564
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.988999
Daily value at risk   -0.014856
dtype: float64


/Users/zlapik/my-drive-zlapik/0-todo/Finrl-Meta/venv3.10/lib/python3.10/site-packages/finrl/meta/preprocessor/yahoodownloader.py:54: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/zlapik/my-drive-zlapik/0-todo/Finrl-Meta/venv3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



<a id='6.2'></a>
## 7.2 BackTestPlot

In [178]:
import pyfolio

%matplotlib inline

baseline_df = get_baseline(ticker="^DJI", start=df_daily_return.loc[0, "date"], end=_TEST_DATA_END)

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns=DRL_strat, benchmark_rets=baseline_returns, set_context=False)

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (251, 8)


/Users/zlapik/my-drive-zlapik/0-todo/Finrl-Meta/venv3.10/lib/python3.10/site-packages/finrl/meta/preprocessor/yahoodownloader.py:54: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/zlapik/my-drive-zlapik/0-todo/Finrl-Meta/venv3.10/lib/python3.10/site-packages/pyfolio/timeseries.py:724: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/Users/zlapik/my-drive-zlapik/0-todo/Finrl-Meta/venv3.10/lib/python3.10/site-packages/pyfolio/plotting.py:648: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



Start date,2021-01-04
End date,2021-12-30
Total months,11
,Backtest
Annual return,19.0%
Cumulative returns,19.0%
Annual volatility,11.7%
Sharpe ratio,1.55
Calmar ratio,2.65
Stability,0.79
Max drawdown,-7.2%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

## Min-Variance Portfolio Allocation

In [144]:
%pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [145]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [146]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [147]:
df.head()

,date,open,high,low,close,adjcp,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033260,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
1,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640541,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
2,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488438,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
3,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034508,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
4,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190567,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


In [148]:
# calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index=range(1), columns=unique_trade_date)
initial_capital = 1000000
portfolio.loc[0, unique_trade_date[0]] = initial_capital

for i in range(len(unique_trade_date) - 1):
    df_temp = df[df.date == unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date == unique_trade_date[i + 1]].reset_index(drop=True)
    # Sigma = risk_models.sample_cov(df_temp.return_list[0])
    # calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    # portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma, weight_bounds=(0, 0.1))
    # minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    # get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    # current capital
    cap = portfolio.iloc[0, i]
    # current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash) / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i + 1] = np.dot(current_shares, next_price)

portfolio = portfolio.T
portfolio.columns = ["account_value"]

In [149]:
portfolio.head()

,account_value
2022-01-03,1000000
2022-01-04,1001477.83443
2022-01-05,997934.935103
2022-01-06,993833.442953
2022-01-07,997084.926838


In [150]:
a2c_cumpod = (df_daily_return.daily_return + 1).cumprod() - 1

In [151]:
min_var_cumpod = (portfolio.account_value.pct_change() + 1).cumprod() - 1

In [152]:
dji_cumpod = (baseline_returns + 1).cumprod() - 1

## Plotly: DRL, Min-Variance, DJIA

In [153]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [154]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

In [155]:
time_ind = pd.Series(df_daily_return.date)

In [156]:
trace0_portfolio = go.Scatter(x=time_ind, y=a2c_cumpod, mode="lines", name="A2C (Portfolio Allocation)")

trace1_portfolio = go.Scatter(x=time_ind, y=dji_cumpod, mode="lines", name="DJIA")
trace2_portfolio = go.Scatter(x=time_ind, y=min_var_cumpod, mode="lines", name="Min-Variance")
# trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
# trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
# trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')

# trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')

# trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
# trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

In [157]:
fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)


fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(family="sans-serif", size=15, color="black"),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2,
    ),
)
# fig.update_layout(legend_orientation="h")
fig.update_layout(
    title={
        #'text': "Cumulative Return using FinRL",
        "y": 0.85,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)
# with Transaction cost
# fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
    #    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="rgba(1,1,0,0)",
    plot_bgcolor="rgba(1, 1, 0, 0)",
    # xaxis_title="Date",
    yaxis_title="Cumulative Return",
    xaxis={"type": "date", "tick0": time_ind[0], "tickmode": "linear", "dtick": 86400000.0 * 80},
)
fig.update_xaxes(showline=True, linecolor="black", showgrid=True, gridwidth=1, gridcolor="LightSteelBlue", mirror=True)
fig.update_yaxes(showline=True, linecolor="black", showgrid=True, gridwidth=1, gridcolor="LightSteelBlue", mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor="LightSteelBlue")

fig.show()